# SAE-96 - Nettoyage Dataset Business

**Objectif:** Charger, analyser et nettoyer le dataset Business de Yelp

**Input:** `data/raw/yelp_academic_dataset_business.json`

**Output:** `data/cleaned/business_clean.parquet`

---

## 1. Imports et Configuration

In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')

print("✅ Imports réussis")

## 2. Chargement des Données

In [ ]:
# Chemins
RAW_DATA_PATH = Path('../data/raw/yelp_academic_dataset_business.json')
CLEANED_DATA_PATH = Path('../data/cleaned/business_clean.parquet')

# Créer le dossier cleaned s'il n'existe pas
CLEANED_DATA_PATH.parent.mkdir(parents=True, exist_ok=True)

print(f"📂 Chargement depuis: {RAW_DATA_PATH}")
print(f"💾 Sauvegarde vers: {CLEANED_DATA_PATH}")

In [ ]:
# Chargement du fichier JSON (format JSON Lines)
business_df = pd.read_json(RAW_DATA_PATH, lines=True)

print(f"✅ Données chargées: {business_df.shape[0]:,} lignes, {business_df.shape[1]} colonnes")
print(f"📊 Taille mémoire: {business_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## 3. Exploration Initiale

In [ ]:
# Aperçu des données
business_df.head()

In [ ]:
# Informations sur les colonnes
business_df.info()

In [ ]:
# Statistiques descriptives
business_df.describe()

## 4. Analyse des Valeurs Manquantes

In [ ]:
# Calcul des valeurs manquantes
missing_data = pd.DataFrame({
    'Colonne': business_df.columns,
    'Valeurs_Manquantes': business_df.isnull().sum(),
    'Pourcentage': (business_df.isnull().sum() / len(business_df) * 100).round(2)
}).sort_values('Pourcentage', ascending=False)

print("📊 Valeurs manquantes par colonne:")
missing_data[missing_data['Valeurs_Manquantes'] > 0]

In [ ]:
# Visualisation des valeurs manquantes
plt.figure(figsize=(12, 6))
missing_cols = missing_data[missing_data['Valeurs_Manquantes'] > 0]
if len(missing_cols) > 0:
    plt.barh(missing_cols['Colonne'], missing_cols['Pourcentage'])
    plt.xlabel('Pourcentage de valeurs manquantes (%)')
    plt.title('Valeurs Manquantes par Colonne - Business Dataset')
    plt.tight_layout()
    plt.show()
else:
    print("✅ Aucune valeur manquante détectée!")

## 5. Nettoyage des Données

In [ ]:
# Copie pour le nettoyage
business_clean = business_df.copy()

print(f"📋 Dataset initial: {len(business_clean):,} lignes")

### 5.1 Suppression des Doublons

In [ ]:
# Vérifier les doublons sur business_id
duplicates = business_clean.duplicated(subset=['business_id'], keep=False)
print(f"🔍 Doublons trouvés: {duplicates.sum()}")

if duplicates.sum() > 0:
    business_clean = business_clean.drop_duplicates(subset=['business_id'], keep='first')
    print(f"✅ Doublons supprimés. Nouvelles lignes: {len(business_clean):,}")
else:
    print("✅ Aucun doublon détecté")

### 5.2 Traitement des Valeurs Manquantes

In [ ]:
# Supprimer les lignes sans business_id (clé primaire)
if business_clean['business_id'].isnull().any():
    before = len(business_clean)
    business_clean = business_clean.dropna(subset=['business_id'])
    print(f"❌ Supprimé {before - len(business_clean)} lignes sans business_id")

# Supprimer les lignes sans nom
if business_clean['name'].isnull().any():
    before = len(business_clean)
    business_clean = business_clean.dropna(subset=['name'])
    print(f"❌ Supprimé {before - len(business_clean)} lignes sans nom")

print(f"\n📊 Dataset après nettoyage: {len(business_clean):,} lignes")

### 5.3 Nettoyage des Colonnes Spécifiques

In [ ]:
# Remplir les valeurs manquantes pour les colonnes numériques
if 'stars' in business_clean.columns:
    business_clean['stars'] = business_clean['stars'].fillna(business_clean['stars'].median())

if 'review_count' in business_clean.columns:
    business_clean['review_count'] = business_clean['review_count'].fillna(0)

# Remplir les valeurs manquantes pour les colonnes textuelles
text_columns = ['city', 'state', 'postal_code', 'address']
for col in text_columns:
    if col in business_clean.columns:
        business_clean[col] = business_clean[col].fillna('Unknown')

print("✅ Valeurs manquantes traitées")

### 5.4 Validation des Données

In [ ]:
# Vérifier que stars est entre 0 et 5
if 'stars' in business_clean.columns:
    invalid_stars = business_clean[(business_clean['stars'] < 0) | (business_clean['stars'] > 5)]
    if len(invalid_stars) > 0:
        print(f"⚠️ {len(invalid_stars)} lignes avec stars invalides")
        business_clean = business_clean[(business_clean['stars'] >= 0) & (business_clean['stars'] <= 5)]
    else:
        print("✅ Toutes les notes sont valides (0-5)")

# Vérifier que review_count est positif
if 'review_count' in business_clean.columns:
    invalid_reviews = business_clean[business_clean['review_count'] < 0]
    if len(invalid_reviews) > 0:
        print(f"⚠️ {len(invalid_reviews)} lignes avec review_count négatif")
        business_clean = business_clean[business_clean['review_count'] >= 0]
    else:
        print("✅ Tous les review_count sont valides (≥0)")

## 6. Statistiques Finales

In [ ]:
print("📊 RÉSUMÉ DU NETTOYAGE")
print("=" * 50)
print(f"Lignes initiales:     {len(business_df):,}")
print(f"Lignes finales:       {len(business_clean):,}")
print(f"Lignes supprimées:    {len(business_df) - len(business_clean):,}")
print(f"Pourcentage conservé: {(len(business_clean) / len(business_df) * 100):.2f}%")
print("=" * 50)

In [ ]:
# Vérification finale des valeurs manquantes
final_missing = business_clean.isnull().sum()
if final_missing.sum() > 0:
    print("\n⚠️ Valeurs manquantes restantes:")
    print(final_missing[final_missing > 0])
else:
    print("\n✅ Aucune valeur manquante dans le dataset nettoyé!")

## 7. Visualisations

In [ ]:
# Distribution des notes
if 'stars' in business_clean.columns:
    plt.figure(figsize=(10, 5))
    business_clean['stars'].hist(bins=20, edgecolor='black')
    plt.xlabel('Note (stars)')
    plt.ylabel('Nombre de commerces')
    plt.title('Distribution des Notes - Business Dataset')
    plt.tight_layout()
    plt.show()

In [ ]:
# Top 10 villes
if 'city' in business_clean.columns:
    plt.figure(figsize=(12, 6))
    top_cities = business_clean['city'].value_counts().head(10)
    top_cities.plot(kind='barh')
    plt.xlabel('Nombre de commerces')
    plt.title('Top 10 Villes - Business Dataset')
    plt.tight_layout()
    plt.show()

## 8. Sauvegarde des Données Nettoyées

In [ ]:
# Sauvegarde en format Parquet (plus efficace que CSV)
business_clean.to_parquet(CLEANED_DATA_PATH, index=False, compression='snappy')

print(f"✅ Données sauvegardées: {CLEANED_DATA_PATH}")
print(f"📊 Taille du fichier: {CLEANED_DATA_PATH.stat().st_size / 1024**2:.2f} MB")

In [ ]:
# Test de rechargement
test_df = pd.read_parquet(CLEANED_DATA_PATH)
print(f"\n✅ Test de rechargement réussi: {len(test_df):,} lignes")
print(f"📋 Colonnes: {list(test_df.columns)}")

---

## ✅ SAE-96 Terminé

**Résultat:** Dataset Business nettoyé et sauvegardé en format Parquet

**Prochaines étapes:**
- SAE-97: Nettoyage Reviews
- SAE-98: Nettoyage Users
- SAE-67: Dashboard Profils Reviewers